Para el problema de la catenaria, de hallar la posición de los eslabones
de una cadena que une los puntos $(X_0, Y_0)$ con $(X_f, Y_f)$, con eslabones de longitud
1, que minimice la energía potencial de la misma (ver Luenberger, página 330).
Hallar condiciones en las que el problema sea factibe, y una aproximación inicla adecuada (que podría ser hallada como punto de partida para un esquema iterativo).  

Si consideramos, como en el libro, los eslabones numerados del 1 al 20 y que el eslabón $i$ tiene un desplazamiento $x_i$ en $x$ y un desplazamiento $y_i$ en $y$, es suficiente que el desplazamiento total en $y$ sea cero mientras que en $x$ sea $16$. Para una aproximación inicial, imagino la cadena tensada hacia abajo por efecto por ejemplo de un peso en el medio de manera tal de que forme un triángulo. Notemos que nos quedan diez eslabones de cada lado, es decir un tripángulo isóceles de lado 10, base 16 y altura 6. Está claro que esta solución no es óptima, pero resulta conveniente para escribir explícitamente ya que los desplazamientos de los eslabones son uniformes, por lo tanto nos queda:

$$x_i = \frac{16}{20}, \ 1 \leq i \leq 20 $$
$$ y_i = \frac{6}{10}, \ 1 \leq i \leq 10 $$
$$ y_i = \frac{-6}{10}, \ 11 \leq i \leq 20 $$


In [40]:
import numpy as np
import plotly.graph_objs as go
x0 = np.array([  # start point: each row is a link (xl, yl, xr, yr)
    0., 0., 0.8, 0.6,
    0.8, 0.6, 1.6, 1.2,
    1.6, 1.2, 2.4, 1.8,
    2.4, 1.8, 3.2, 2.4,
    3.2, 2.4, 4., 3.,
    4., 3., 4.8, 3.6,
    4.8, 3.6, 5.6, 4.2,
    5.6, 4.2, 6.4, 4.8,
    6.4, 4.8, 7.2, 5.4,
    7.2, 5.4, 8., 6.,
    8., 6., 8.8, 5.4, 
    8.8, 5.4, 9.6, 4.8,
    9.6, 4.8, 10.4, 4.2,
    10.4, 4.2, 11.2, 3.6,
    11.2, 3.6, 12., 3.,
    12., 3., 12.8, 2.4,
    12.8, 2.4, 13.6, 1.8,
    13.6, 1.8, 14.4, 1.2,
    14.4, 1.2, 15.2, 0.6,
    15.2, 0.6, 16, 0
])
x = x0[0::2]  # all x's coords
y = x0[1::2]  # all y's coords
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, mode='markers'))
fig.update_yaxes(range=[6.5, -0.5])
fig.show()

In [14]:
from scipy.constants import g
def obj_function(x):
    """Función objetivo a minimizar.
    
    Parámetros
    ----------
    x : numpy.array
        Vector de 80 dimensiones representando las posiciones de los 20 eslabones
    
    Resultado
    ---------
    rv : float
        Energía potencial total de la cadena según la configuración dada en `x`.

    """
    y_coords = x[1::2]  # energia potencial se calcula solo con las coordenadas y
    rv = 0.5 * g * sum(y_coords)
    return rv

def link_size(x, l):
    """Objective function to minimize.
    
    Parámetros
    ----------
    x : numpy.array
        Vector de 80 dimensiones representando las posiciones de los 20 eslabones
    l : int
        Indice del eslabón, debe ser uno de 0, 1, 2, ..., 19
    
    Resultado
    ---------
    rv : float
        Length of the link `l` given by the configuration in `x`.
    
    """
    i = 4*l
    rv = (x[i] - x[i+2])**2 + (x[i+1] - x[i+3])**2
    return round(rv, 3)

Para ser congruente con el indexado de python, numero las coordenadas de $x$ empezando en cero, y les asigno el siguiente orden arbitrario:
$$x = (x_0^l, y_0^l, x_0^r, y_0^r, x_1^l, y_1^l, x_1^r, y_1^r, ..., x_{19}^l, y_{19}^l, x_{19}^r, y_{19}^r)$$

In [17]:
cons = [ # 4 restricciones de los eslabones inicial y final
    {'type': 'eq', 'fun': lambda x: x[0]},
    {'type': 'eq', 'fun': lambda x: x[1]},
    {'type': 'eq', 'fun': lambda x: x[79]},
    {'type': 'eq', 'fun': lambda x: x[78] - 16}
]
for i in range(2, 78, 4):  # restricciones que sujetan cada eslabon con el siguiente
    # 19 para las x's y 19 para las y's.
#     print(f'x({i}) = x({i+2})')
#     print(f'y({i+1}) = y({i+3})')
    cons.append({'type': 'eq', 'fun': lambda x, i=i: x[i] - x[i+2]})
    cons.append({'type': 'eq', 'fun': lambda x, i=i: x[i+1] - x[i+3]})

for i in range(20):
    # 20 restricciones que fijan el tamaño de los eslabones
    cons.append({'type': 'ineq', 'fun': lambda x, i=i: link_size(x, i) - 1})

cons.append({'type': 'eq', 'fun': lambda x: sum(x[2::4] - x[0::4]) - 16})
print(f'Total: {len(cons)} restricciones.')

Total: 63 restricciones.


In [41]:
# factibilidad de x0
print(f"Factibilidad: {(np.array([h['fun'](x0) for h in cons]) == 0).all()}")

Factibilidad: True


In [22]:
from scipy.optimize import minimize
solution = minimize(obj_function, x0, constraints=cons)
solution.success, solution.status, solution.message

(False, 8, 'Positive directional derivative for linesearch')

In [23]:
solution
x_sol = solution.x[0::2]
y_sol = solution.x[1::2]
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_sol, y=y_sol, mode='markers'))
fig.show()